In [1]:
# 🔹 1. Imports
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))  # Ajuste de ruta si estás dentro de notebooks/


from scripts.pc_smote import PCSMOTE
from datasets.landsat8_lujan_campo_20250531_path225_row084_SR.preprocess_landsat import cargar_dataset_landsat

# 🔹 2. Cargar datos Landsat
path_bandas = "codigo/datasets/landsat8_lujan_campo_20250531_path225_row084_SR"
path_mascara = f"{path_bandas}/etiquetas_mask.tif"

X, y = cargar_dataset_landsat(path_bandas, path_mascara)

print(f"🟦 X shape: {X.shape}")
print(f"🟥 y distribution:\n{pd.Series(y).value_counts()}")

# 🔹 3. Aplicar técnica de sobremuestreo
samplers = {
    "SMOTE": SMOTE(random_state=42),
    "ADASYN": ADASYN(random_state=42),
    "BorderlineSMOTE": BorderlineSMOTE(random_state=42),
    "PC-SMOTE": PCSMOTE(k_neighbors=5, radio_densidad=0.5, percentil_dist=75)
}

# 🔹 4. Entrenamiento y evaluación
from sklearn.model_selection import train_test_split

resultados = []

for nombre, sampler in samplers.items():
    print(f"\n🔄 {nombre}...")

    X_res, y_res = sampler.fit_resample(X, y)
    X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3, stratify=y_res, random_state=42)

    clf = RandomForestClassifier(random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    print(classification_report(y_test, y_pred, digits=4))
    print("Confusion matrix:")
    print(confusion_matrix(y_test, y_pred))

    resultados.append({
        "técnica": nombre,
        "accuracy": clf.score(X_test, y_test),
        "n_clases": len(np.unique(y_res))
    })

# 🔹 5. Comparar resultados
df_resultados = pd.DataFrame(resultados)
display(df_resultados)


FileNotFoundError: No se encontró la banda SR_B2 en codigo\datasets\landsat8_lujan_campo_20250531_path225_row084_SR

In [2]:
from pathlib import Path

path_bandas = Path("codigo/datasets/landsat8_lujan_campo_20250531_path225_row084_SR")

for tif in path_bandas.glob("*.TIF"):
    print(tif.name)
